In [2]:
#r "nuget:APSToolkit"

Installed Packages APSToolkit, 1.0.4

In [3]:
using APSToolkit;
using Autodesk.Forge;
using APSToolkit.Database;

## Auth

In [7]:
using APSToolkit.Auth;
var token = Authentication.Get2LeggedToken().Result;

In [14]:
// please visit tutorial : 03.Explore Versions Item and Derivative Urn to get the urn
string urn = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLk9kOHR4RGJLU1NlbFRvVmcxb2MxVkE_dmVyc2lvbj0z";

## Database Reader

In [8]:
var DbReader = new DbReader(urn, token);

#### Read Query Database ElementId

In [13]:
using System.Data;
string sqlQuery = @"
            SELECT _objects_id.id AS dbId, _objects_id.external_id AS externalId, 
                   _objects_attr.name AS name,_objects_attr.display_name AS propName , 
                   _objects_val.value AS propValue
            FROM _objects_eav
                INNER JOIN _objects_id ON _objects_eav.entity_id = _objects_id.id
                INNER JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
                INNER JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
            WHERE name = 'ElementId'
            ";
// update for get
DataTable dataTable = DbReader.ExecuteQuery(sqlQuery);
// see how many column and row in the table
Console.WriteLine(dataTable.Rows.Count);
Console.WriteLine(dataTable.Columns.Count);

4017
5


In [15]:
#r "nuget:Microsoft.Data.Analysis"

Installed Packages microsoft.data.analysis, 0.20.1

In [30]:
using Microsoft.Data.Analysis;
public DataFrame ToDataFrame(DataTable dataTable)
    {
        DataFrame dataFrame = new DataFrame();
        
        foreach (DataColumn column in dataTable.Columns)
        {
            // get values from column cast as string
            string[] values = dataTable.AsEnumerable().Select(r => r.Field<object>(column.ColumnName)?.ToString()).ToArray();
            DataFrameColumn dataFrameColumn = DataFrameColumn.Create(column.ColumnName, values);
            dataFrame.Columns.Add(dataFrameColumn);
        }
        return dataFrame;
        return dataFrame;
    }
var df = ToDataFrame(dataTable);
df = df.Head(5);
df

index,dbId,externalId,name,propName,propValue
0,40,9693c920-050e-4443-9b38-2491295576d1-00000000,ElementId,<null>,0
1,42,9693c920-050e-4443-9b38-2491295576d1-00000001,ElementId,<null>,1
2,43,9693c920-050e-4443-9b38-2491295576d1-00000002,ElementId,<null>,2
3,44,9693c920-050e-4443-9b38-2491295576d1-00000003,ElementId,<null>,3
4,45,9693c920-050e-4443-9b38-2491295576d1-00000004,ElementId,<null>,4
